In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import math

In [2]:
def discrimination(data, target, sens, expl, max_corr=.1):
    # target
    # sens: sensitive attribute
    # expl: explanatory attribute(s), str or list
    group_priv = data[data[sens]==2]
    group_prot = data[data[sens]==1]
    n_priv = group_priv.shape[0]
    n_prot = group_prot.shape[0]
    
    D_all = np.sum(group_priv[target]==1)/n_priv - np.sum(group_prot[target] == 1)/n_prot
    print('Total discrimination: %.3f' % D_all)
    
    # multiple explanatory attributes
    if isinstance(expl, list):
        high_corr = list(data.columns[np.abs(data.corr()[sens].sort_values()) > max_corr])
        high_corr = []
        for e in expl: 
            if e in high_corr: 
                print(e, 'is highly correlated with', sens)
        expl = [e for e in expl if e not in high_corr]
        data_expl = pd.Series(KMeans(n_clusters=4).fit(data[expl]).labels_)
    else:
        data_expl = data[expl]
    
    data_expl_priv = data_expl[data[sens]==2]
    data_expl_prot = data_expl[data[sens]==1]
        
    expl_values = data_expl.unique()
    D_expl = 0 
    
    for e_i in expl_values:
        P_star = (np.sum((group_priv[target]==1) & (data_expl_priv == e_i))/(np.sum(data_expl_priv == e_i)) + 
                  np.sum((group_prot[target]==1) & (data_expl_prot == e_i))/(np.sum(data_expl_prot == e_i)))/2
        if np.isnan(P_star): 
            P_star = 0
        D_expl += (np.sum(data_expl_priv == e_i)/len(data_expl_priv) -  
                   np.sum(data_expl_prot == e_i)/len(data_expl_prot)) * P_star
        print('Favorable group prob, non-favorable group prob, Correct prob: ', 
             np.sum((group_priv[target]==1) & (data_expl_priv == e_i))/(np.sum(data_expl_priv == e_i)), 
             np.sum((group_prot[target]==1) & (data_expl_prot == e_i))/(np.sum(data_expl_prot == e_i)), 
             P_star)
#         print('Total samples, postive favourable group samples, positive non-favourable group samples: ', 
#              (np.sum(data_expl_priv == e_i))+(np.sum(data_expl_prot == e_i)), 
#              np.sum((group_priv[target]==1) & (data_expl_priv == e_i)), 
#              np.sum((group_prot[target]==1) & (data_expl_prot == e_i)))
        
    print('Discrimination explainable by %s: %.3f' % (', '.join(expl), D_expl))
    
    D_illegal = D_all - D_expl
    print('Unexplainable discrimination: %.3f' % D_illegal)
    
    return (D_all, D_expl, D_illegal)

In [3]:
# local massaging
def local_massaging(data, target, sens, expl, t_feature, max_corr=.1):
    # expl: explanatory attribute(s), str or list
    # t_feature: training feature for the ranker
    group_priv = data[data[sens]==2]
    group_prot = data[data[sens]==1]
    n_priv = group_priv.shape[0]
    n_prot = group_prot.shape[0]
    
    D_all = np.sum(group_priv[target]==1)/n_priv - np.sum(group_prot[target] == 1)/n_prot
    print('Total discrimination: %.3f' % D_all)
    
    # multiple explanatory attributes
    if isinstance(expl, list):
        high_corr = list(data.columns[np.abs(data.corr()[sens].sort_values()) > max_corr])
        high_corr = []
        for e in expl: 
            if e in high_corr: 
                print(e, 'is highly correlated with', sens)
        expl = [e for e in expl if e not in high_corr]
        data_expl = pd.Series(KMeans(n_clusters=4).fit(data[expl]).labels_)
    else:
        data_expl = data[expl]
    
    data_expl_priv = data_expl[data[sens]==2]
    data_expl_prot = data_expl[data[sens]==1]
    
    data['cluster_label'] = data_expl
        
    expl_values = data_expl.unique()
    D_expl = 0 
    e_subgroups = []
    
    for e_i in expl_values:
        fav_prob = np.sum((group_priv[target]==1) & (data_expl_priv == e_i))/(np.sum(data_expl_priv == e_i))
        non_fav_prob = np.sum((group_prot[target]==1) & (data_expl_prot == e_i))/(np.sum(data_expl_prot == e_i))
        P_star = (fav_prob + non_fav_prob)/2
        if np.isnan(P_star): 
            P_star = 0
            print('One group is absent for this explainable value')
        D_expl += (np.sum(data_expl_priv == e_i)/len(data_expl_priv) -  
                   np.sum(data_expl_prot == e_i)/len(data_expl_prot)) * P_star
        print('Favorable group prob, non-favorable group prob, Correct prob: ', 
             fav_prob, non_fav_prob, P_star)
        print('Total samples, postive favourable group samples, positive non-favourable group samples: ', 
             (np.sum(data_expl_priv == e_i))+(np.sum(data_expl_prot == e_i)), 
             np.sum((group_priv[target]==1) & (data_expl_priv == e_i)), 
             np.sum((group_prot[target]==1) & (data_expl_prot == e_i)))
        
        # get sub group of current explainable value
        sub_grp = data[data_expl == e_i].copy()
        sub_grp.reset_index(inplace=True, drop=True)
        
        # calculate the number of samples that need to be flipped
        delta_priv = int(round(abs(np.sum((sub_grp[sens]==2) & (sub_grp[target]==1)) - 
                             (np.sum(sub_grp[sens]==2) * P_star))))
        delta_prot = int(round(abs(np.sum((sub_grp[sens]==1) & (sub_grp[target]==1)) - 
                             (np.sum(sub_grp[sens]==1) * P_star))))
        if P_star == 0:
            delta_priv = 0
            delta_prot = 0
        print('Required number of flipping samples for favourable and non-favourable group: ', 
              delta_priv, delta_prot)
        
        # get scores and rank
        score, ranking =  get_ranking_score(sub_grp, t_feature)
        sub_grp['score'] = score
        sub_grp.sort_values('score', ascending=False, inplace=True)
        
        # flip the label for last samples in each group
        sub_grp_priv_pos = sub_grp[(sub_grp[sens]==2) & (sub_grp[target]==1)].copy().reset_index(drop=True)
        sub_grp_prot_pos = sub_grp[(sub_grp[sens]==1) & (sub_grp[target]==1)].copy().reset_index(drop=True)
        sub_grp_priv_neg = sub_grp[(sub_grp[sens]==2) & (sub_grp[target]==0)].copy().reset_index(drop=True)
        sub_grp_prot_neg = sub_grp[(sub_grp[sens]==1) & (sub_grp[target]==0)].copy().reset_index(drop=True)
        if P_star != 0:
            if fav_prob > non_fav_prob:
                sub_grp_priv_pos.iloc[-delta_priv:, 0] = 0
                sub_grp_prot_neg.iloc[:delta_prot, 0] = 1
            else:
                sub_grp_priv_neg.iloc[:delta_priv, 0] = 1
                sub_grp_prot_pos.iloc[-delta_prot:, 0] = 0
            
#         print('Favourable group prob, Non-favourable group prob: ', 
#              np.sum(sub_grp_priv[target]==1)/len(sub_grp_priv), 
#              np.sum(sub_grp_prot[target]==1)/len(sub_grp_prot))
        print('********************************')
            
        # append to arrays
        e_subgroups.append(sub_grp_priv_pos)
        e_subgroups.append(sub_grp_prot_pos)
        e_subgroups.append(sub_grp_priv_neg)
        e_subgroups.append(sub_grp_prot_neg)
    
    revised_df = pd.concat(e_subgroups, ignore_index=False)
    revised_df.reset_index(inplace=True, drop=True)
    print(revised_df.info())
        
#     print('Discrimination explainable by %s: %.3f' % (', '.join(expl), D_expl))
    
#     D_illegal = D_all - D_expl
#     print('Unexplainable discrimination: %.3f' % D_illegal)
    
    return revised_df

In [4]:
# local massaging
def local_preferential_samping(data, target, sens, expl, t_feature, max_corr=.1):
    # expl: explanatory attribute(s), str or list
    # t_feature: training feature for the ranker
    group_priv = data[data[sens]==2]
    group_prot = data[data[sens]==1]
    n_priv = group_priv.shape[0]
    n_prot = group_prot.shape[0]
    
    D_all = np.sum(group_priv[target]==1)/n_priv - np.sum(group_prot[target] == 1)/n_prot
    print('Total discrimination: %.3f' % D_all)
    
    # multiple explanatory attributes
    if isinstance(expl, list):
        high_corr = list(data.columns[np.abs(data.corr()[sens].sort_values()) > max_corr])
        high_corr = []
        for e in expl: 
            if e in high_corr: 
                print(e, 'is highly correlated with', sens)
        expl = [e for e in expl if e not in high_corr]
        data_expl = pd.Series(KMeans(n_clusters=4).fit(data[expl]).labels_)
    else:
        data_expl = data[expl]
    
    data_expl_priv = data_expl[data[sens]==2]
    data_expl_prot = data_expl[data[sens]==1]
    
    data['cluster_label'] = data_expl
        
    expl_values = data_expl.unique()
    D_expl = 0 
    e_subgroups = []
    
    for e_i in expl_values:
        fav_prob = np.sum((group_priv[target]==1) & (data_expl_priv == e_i))/(np.sum(data_expl_priv == e_i))
        non_fav_prob = np.sum((group_prot[target]==1) & (data_expl_prot == e_i))/(np.sum(data_expl_prot == e_i))
        P_star = (fav_prob + non_fav_prob)/2
        if np.isnan(P_star): 
            P_star = 0
            print('One group is absent for this explainable value')
        D_expl += (np.sum(data_expl_priv == e_i)/len(data_expl_priv) -  
                   np.sum(data_expl_prot == e_i)/len(data_expl_prot)) * P_star
        print('Favorable group prob, non-favorable group prob, Correct prob: ', 
             fav_prob, non_fav_prob, P_star)
        print('Total samples, postive favourable group samples, positive non-favourable group samples: ', 
             (np.sum(data_expl_priv == e_i))+(np.sum(data_expl_prot == e_i)), 
             np.sum((group_priv[target]==1) & (data_expl_priv == e_i)), 
             np.sum((group_prot[target]==1) & (data_expl_prot == e_i)))
        
        # get sub group of current explainable value
        sub_grp = data[data_expl == e_i].copy()
        sub_grp.reset_index(inplace=True, drop=True)
        
        # calculate the number of samples that need to be flipped
        delta_priv = int(round(abs(np.sum((sub_grp[sens]==2) & (sub_grp[target]==1)) - 
                             (np.sum(sub_grp[sens]==2) * P_star))))
        delta_prot = int(round(abs(np.sum((sub_grp[sens]==1) & (sub_grp[target]==1)) - 
                             (np.sum(sub_grp[sens]==1) * P_star))))
        if P_star == 0:
            delta_priv = 0
            delta_prot = 0
        print('Required number of flipping samples for favourable and non-favourable group: ', 
              delta_priv, delta_prot)
        
        # get scores and rank
        score, ranking =  get_ranking_score(sub_grp, t_feature)
        sub_grp['score'] = score
        sub_grp.sort_values('score', ascending=False, inplace=True)
        
        # flip the label for last samples in each group
        sub_grp_priv_pos = sub_grp[(sub_grp[sens]==2) & (sub_grp[target]==1)].copy().reset_index(drop=True)
        sub_grp_prot_pos = sub_grp[(sub_grp[sens]==1) & (sub_grp[target]==1)].copy().reset_index(drop=True)
        sub_grp_priv_neg = sub_grp[(sub_grp[sens]==2) & (sub_grp[target]==0)].copy().reset_index(drop=True)
        sub_grp_prot_neg = sub_grp[(sub_grp[sens]==1) & (sub_grp[target]==0)].copy().reset_index(drop=True)
        if P_star != 0:
            if fav_prob > non_fav_prob:
                sub_grp_priv_pos.drop(range(len(sub_grp_priv_pos)-int(0.5*delta_priv), len(sub_grp_priv_pos)))
                sub_grp_priv_neg.append(sub_grp_priv_neg.iloc[:int(0.5*delta_priv)])
                sub_grp_prot_pos.append(sub_grp_prot_pos.iloc[-int(0.5*delta_prot):])
                sub_grp_prot_neg.drop(range(int(0.5*delta_prot)))
            else:
                sub_grp_prot_pos.drop(range(len(sub_grp_prot_pos)-int(0.5*delta_prot), len(sub_grp_prot_pos)))
                sub_grp_prot_neg.append(sub_grp_prot_neg.iloc[:int(0.5*delta_prot)])
                sub_grp_priv_pos.append(sub_grp_priv_pos.iloc[-int(0.5*delta_priv):])
                sub_grp_priv_neg.drop(range(int(0.5*delta_priv)))
            
#         print('Favourable group prob, Non-favourable group prob: ', 
#              np.sum(sub_grp_priv[target]==1)/len(sub_grp_priv), 
#              np.sum(sub_grp_prot[target]==1)/len(sub_grp_prot))
        print('********************************')
            
        # append to arrays
        e_subgroups.append(sub_grp_priv_pos)
        e_subgroups.append(sub_grp_prot_pos)
        e_subgroups.append(sub_grp_priv_neg)
        e_subgroups.append(sub_grp_prot_neg)
    
    revised_df = pd.concat(e_subgroups, ignore_index=False)
    revised_df.reset_index(inplace=True, drop=True)
    print(revised_df.info())
        
#     print('Discrimination explainable by %s: %.3f' % (', '.join(expl), D_expl))
    
#     D_illegal = D_all - D_expl
#     print('Unexplainable discrimination: %.3f' % D_illegal)
    
    return revised_df

In [5]:
# Train a logistic model to get ranking and score (probability of being positive) for each smaple
def get_ranking_score(data, features):
    # features: features used for training the ranker
    # return: two arrays, ranking and scores
    lr = LogisticRegression().fit(data[features], data['Creditability'])
#     coefs = dict(zip(X.columns, np.round(list(lr.coef_[0]), 2)))
    y_prob = lr.predict_proba(data[features])[:, 1]
    ranking = np.argsort(y_prob)[::-1]
#     print(y_prob)
#     print(ranking)
#     print(y_prob[ranking])
    return y_prob, ranking

In [6]:
credit_df = pd.read_csv('./resampled_nation_gender.csv')
print(credit_df.columns)

male_idx = (credit_df['Sex & Marital Status']==1) | (credit_df['Sex & Marital Status']==3) | \
                 (credit_df['Sex & Marital Status']==4)
female_idx = (credit_df['Sex & Marital Status']==2) | (credit_df['Sex & Marital Status']==5)
native_idx = (credit_df['Foreign Worker']==2)
foreign_idx = (credit_df['Foreign Worker']==1)

# insert a column of gender, 1 female, 2 female
# credit_df.insert(loc=len(credit_df.columns), column='gender', value=1)
# credit_df.loc[male_idx, 'gender'] = 2

legal = credit_df.columns[ [1, 3, 5, 6, 8, 10, 12, 14, 16 ] ]
maybe = credit_df.columns[ [1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 14, 15, 16, 17 ] ]

# discrimination(credit_df, 'Creditability', 'gender', list(maybe))
# discrimination(credit_df, 'Creditability', 'gender', 'Occupation')
# get_ranking_score(credit_df, list(maybe))
discrimination(credit_df, 'Creditability', 'Foreign Worker', list(maybe))
# revised_df = local_massaging(credit_df, 'Creditability', 'Foreign Worker', list(maybe), list(maybe))
revised_df = local_preferential_samping(credit_df, 'Creditability', 'Foreign Worker', list(maybe), list(maybe))
# revised_df[0:3]
discrimination(revised_df, 'Creditability', 'Foreign Worker', 'cluster_label')

Index(['Creditability', 'Account Balance', 'Duration of Credit (month)',
       'Payment Status of Previous Credit', 'Purpose', 'Credit Amount',
       'Value Savings/Stocks', 'Length of current employment',
       'Instalment per cent', 'Sex & Marital Status', 'Guarantors',
       'Duration in Current address', 'Most valuable available asset',
       'Age (years)', 'Concurrent Credits', 'Type of apartment',
       'No of Credits at this Bank', 'Occupation', 'No of dependents',
       'Telephone', 'Foreign Worker', 'gender'],
      dtype='object')
Total discrimination: 0.166
Favorable group prob, non-favorable group prob, Correct prob:  0.9789029535864979 0.684981684981685 0.8319423192840915
Favorable group prob, non-favorable group prob, Correct prob:  0.9230769230769231 0.5846153846153846 0.7538461538461538
Favorable group prob, non-favorable group prob, Correct prob:  0.9635036496350365 0.732484076433121 0.8479938630340788
Favorable group prob, non-favorable group prob, Correct prob

(0.16599999999999993, -0.08261960150981777, 0.2486196015098177)

Measuring the discrimination in prediction using logistic regression

In [7]:
# credit_df['Credit Amount'] = np.log(credit_df['Credit Amount'])
# X = credit_df
# y = credit_df['Creditability']
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [8]:
revised_df['Credit Amount'] = np.log(revised_df['Credit Amount'])
X = revised_df
y = revised_df['Creditability']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [9]:
lr = LogisticRegression().fit(X_train[maybe], y_train)
# coefs = dict(zip(X.columns, np.round(list(lr.coef_[0]), 2)))
y_pred = lr.predict(X_test[maybe])
print('Accuracy:', accuracy_score(y_test, y_pred))
data_test = X_test.copy()
print(data_test.shape)
data_test.reset_index(inplace=True)
data_test['Creditability'] = y_pred
# discrimination(data_test, 'Creditability', 'gender', list(maybe))
discrimination(data_test, 'Creditability', 'Foreign Worker', list(maybe))

c_female = np.sum(data_test['gender']==1)
c_male = np.sum(data_test['gender']==2)
c_credible_female = np.sum((data_test['gender']==1) & (data_test['Creditability']==1))
c_credible_male = np.sum((data_test['gender']==2) & (data_test['Creditability']==1))
c_foreign = np.sum(data_test['Foreign Worker']==1)
c_native = np.sum(data_test['Foreign Worker']==2)
c_credible_foreign = np.sum((data_test['Foreign Worker']==1) & (data_test['Creditability']==1))
c_credible_native = np.sum((data_test['Foreign Worker']==2) & (data_test['Creditability']==1))

print('Female and male: ', c_female, c_male)
print('Credible Female and male: ', c_credible_female, c_credible_male, 
      c_credible_female/c_female, c_credible_male/c_male)
print('Foreign and native: ', c_foreign, c_native)
print('Credible Foreign and native: ', c_credible_foreign, c_credible_native,
      c_credible_foreign/c_foreign, c_credible_native/c_native)

Accuracy: 0.856
(250, 24)
Total discrimination: 0.020
Favorable group prob, non-favorable group prob, Correct prob:  0.0 0.1111111111111111 0.05555555555555555
Favorable group prob, non-favorable group prob, Correct prob:  1.0 0.9833333333333333 0.9916666666666667
Favorable group prob, non-favorable group prob, Correct prob:  0.9473684210526315 0.7272727272727273 0.8373205741626795
Favorable group prob, non-favorable group prob, Correct prob:  nan 0.6470588235294118 0
Discrimination explainable by Account Balance, Duration of Credit (month), Payment Status of Previous Credit, Purpose, Credit Amount, Value Savings/Stocks, Instalment per cent, Guarantors, Duration in Current address, Most valuable available asset, Concurrent Credits, Type of apartment, No of Credits at this Bank, Occupation: 0.023
Unexplainable discrimination: -0.002
Female and male:  125 125
Credible Female and male:  89 110 0.712 0.88
Foreign and native:  141 109
Credible Foreign and native:  111 88 0.7872340425531915 

/home/xgli/anaconda3/envs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in long_scalars
/home/xgli/anaconda3/envs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in long_scalars


In [10]:
df = pd.DataFrame(data={'col1':[1, 2, 3]})
# df.loc[df.index[True, False, True], 'col1'] = 10
idx = np.array(df.index)
idx[[True, False, True]].shape

(2,)